In [1]:
#r "nuget: Langchain, *-*"
#r "nuget: Langchain.Core,*-*"
#r "nuget: LangChain.DocumentLoaders.Pdf, *-*"
#r "nuget: LangChain.Databases.Sqlite, *-*"
#r "nuget: LangChain.Providers.Ollama, *-*"
#r "nuget: LangChain.Splitters.CSharp, *-*"
#r "nuget: Ollama, *-*"

Installed Packages Langchain, 0.14.1-dev.30 Langchain.Core, 0.14.1-dev.30 LangChain.Databases.Sqlite, 0.14.1-dev.30 LangChain.DocumentLoaders.Pdf, 0.14.1-dev.30 LangChain.Providers.Ollama, 0.14.1-dev.30 LangChain.Splitters.CSharp, 0.14.1-dev.30 Ollama, 1.0.0

In [2]:
using LangChain.Databases.Sqlite;
using LangChain.DocumentLoaders;
using LangChain.Providers.Ollama;
using LangChain.Extensions;
using Ollama;

In [3]:
var provider = new OllamaProvider(options: new RequestOptions
{
    Stop = ["\n"],
    Temperature = 0.0f,
});
var embeddingModel = new OllamaEmbeddingModel(provider, id: "all-minilm");
//var embeddingModel = new OllamaEmbeddingModel(provider, id: "nomic-embed-text");
//var embeddingModel = new OllamaEmbeddingModel(provider, id: "mxbai-embed-large")
var llm = new OllamaChatModel(provider, id: "llama3");
var vectorDatabase = new SqLiteVectorDatabase(dataSource: "vectors.db");

In [4]:
var vectorCollection = await vectorDatabase.AddDocumentsFromAsync<PdfPigPdfLoader>(
    embeddingModel, // Used to convert text to embeddings
    dimensions: 1536, // Should be 1536 for TextEmbeddingV3SmallModel
    //dimensions: 384, 
    dataSource: DataSource.FromUrl("https://canonburyprimaryschool.co.uk/wp-content/uploads/2016/01/Joanne-K.-Rowling-Harry-Potter-Book-1-Harry-Potter-and-the-Philosophers-Stone-EnglishOnlineClub.com_.pdf"),
    collectionName: "harrypotter", // Can be omitted, use if you want to have multiple collections
    textSplitter: null,
    behavior: AddDocumentsToDatabaseBehavior.JustReturnCollectionIfCollectionIsAlreadyExists);
     

In [5]:
const string question = "What is Harry's Address?";
var similarDocuments = await vectorCollection.GetSimilarDocuments(embeddingModel, question, amount: 5);
// Use similar documents and LLM to answer the question
var answer = await llm.GenerateAsync(
    $"""
     Use the following pieces of context to answer the question at the end.
     If the answer is not in context then just say that you don't know, don't try to make up an answer.
     Keep the answer as short as possible.

     {similarDocuments.AsString()}

     Question: {question}
     Helpful Answer:
     """).ConfigureAwait(false);

Console.WriteLine($"LLM answer: {answer}");
Console.WriteLine("Similar Documents:");
foreach(var document in similarDocuments)
{
    Console.WriteLine(document);
}

LLM answer:  The address written on the letter is: Mr H. Potter The Cupboard under the Stairs 4 Privet Drive Little Whinging Surrey
Similar Documents:
(PageContent='42 HARRY  POTTER   they’re famous. You’re famous.’ ‘What? My – my mum and dad weren’t famous, were they?’ ‘Yeh don’ know ... yeh don’ know ...’ Hagrid ran his fingers through his hair, fixing Harry with a bewildered stare. ‘Yeh don’ know what yeh are?’ he said finally. Uncle Vernon suddenly found his voice. ‘Stop!’ he commanded. ‘Stop right there, sir! I forbid you to tell the boy anything!’ A braver man than Vernon Dursley would have quailed under the furious look Hagrid now gave him; when Hagrid spoke, his every syllable trembled with rage. ‘You never told him? Never told him what was in the letter Dumbledore left fer him? I was there! I saw Dumbledore leave it, Dursley! An’ you’ve kept it from him all these years?’ ‘Kept what from me?’ said Harry eagerly. ‘STOP! I FORBID YOU!’ yelled Uncle Vernon in panic. Aunt Petunia g